In [1]:
# Import dependecies

import nltk
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

In [2]:
import numpy as np
import string
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [3]:
from google.colab import files

files.upload()

Saving utils.py to utils.py


{'utils.py': b'# -*- coding: utf-8 -*-\n"""utils.ipynb\n\nAutomatically generated by Colaboratory.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/14Fp_ARD8EDcQ4_AtdG3jCXoc9RODzHTV\n"""\n"""Import Dependencies"""\n\nimport numpy as np\nimport string\nimport re\n\nimport nltk\nfrom nltk.corpus import stopwords\nfrom nltk.stem import PorterStemmer\nfrom nltk.tokenize import TweetTokenizer\n\n"""Preprocessing function"""\n\ndef process_tweet(tweet):\n  """\n  Input: a string\n  output: list of words\n  """\n\n  stemmer = PorterStemmer()\n  tokenizer = TweetTokenizer(preserve_case = False, reduce_len = True, strip_handles = True)\n  stopword_english = stopwords.words(\'english\')\n\n  # remove retweet sign, stock tickers, urls, #\n\n  tweet = re.sub(r\'^RT[\\s]+\', \'\', tweet)\n  tweet = re.sub(r\'^\\$\\w*\', \'\', tweet)\n  tweet = re.sub(r\'https?://[^\\s\\n\\r]+\', \'\', tweet)\n  tweet = re.sub(r\'#\', \'\', tweet)\n\n  tokens = tokenizer.tokenize(tweet)\n\

In [4]:
import utils
from utils import process_tweet, count_freqs

In [5]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [7]:
# split the data into two pieces, one for training and one for testing (validation set) 
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [8]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [9]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [10]:
# create frequency dictionary
freqs = count_freqs(train_x, train_y)

# check the output
print("type = " + str(type(freqs)))
print("length = " + str(len(freqs.keys())))

type = <class 'dict'>
length = 11427


Logistic regression model

In [11]:
# create sigmoid function

def sigmoid(z):
  h = 1 / (1 + np.exp(-z))
  return h

In [12]:
# create gradient descent function

def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: feature matrix; dim=(m,n+1)
        y: labels; dim=(m,1)
        theta: weight vector; dimension=(n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    '''
    # the number of rows in matrix x
    m = x.shape[0]
    
    for i in range(0, num_iters):

        # the dot product of x and theta
        z = np.dot(x,theta)
        
        # sigmoid of z
        h = sigmoid(z)
        
        # cost function
        J = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))    

        # update theta
        theta = theta - (alpha/m) * np.dot(x.transpose(),(h-y))
        
    J = float(J)
    return J, theta

In [13]:
# extracting the features of one tweet

def extract_features(tweet, freqs):

    # process the tweet
    word_l = process_tweet(tweet)
    
    # feature vector [bias, pos, neg] = [0,0,0]
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
       
    # loop through each word in the list of words
    for word in word_l:
        x[0,1] += freqs.get((word, 1.0),0)  # get the frequency value for dictionary and add to it
        x[0,2] += freqs.get((word, 0.0),0)
        
    assert(x.shape == (1, 3))
    return x

In [16]:
# train the model

# create matrix with features from all the tweets
X = np.zeros((len(train_x), 3))

for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

Y = train_y

J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(J)
print([round(t, 8) for t in np.squeeze(theta)])

0.22521315349223694
[6e-08, 0.0005382, -0.0005583]


In [17]:
# test the model
def test_logistic_regression(test_x, test_y, freqs, theta):
   
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)
    
    return accuracy

In [19]:
# Predict whether a tweet is positive or negative

def predict_tweet(tweet, freqs, theta):
    # extract the features in x 
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x,theta))
        
    return y_pred

In [21]:
test_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print("Accuracy = " + str(test_accuracy))

Accuracy = 0.995


In [24]:
# Make prediction with a made up tweet
my_tweet = 'The more I use @salesforce the more I dislike it. It is slow and full of bugs.'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['use', 'dislik', 'slow', 'full', 'bug']
[[0.49973361]]
Negative sentiment


In [23]:
# Make prediction with a made up tweet
my_tweet = 'This beat drop is the bomb! #edm #music #dubstep #newrelease'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['beat', 'drop', 'bomb', 'edm', 'music', 'dubstep', 'newreleas']
[[0.50149899]]
Positive sentiment
